### Define the Python Environment

In [ ]:
# Load the Python environment.
from collections import OrderedDict

import matplotlib.pyplot as plt
import numpy
from ioapps import netcdf4_interface
from IPython.display import HTML, display
from tabulate import tabulate
from tools import parser_interface

## Jupyter Notebook: tcwnmsi.ipynb

Email: henry.winterbottom@noaa.gov

This program is free software: you can redistribute it and/or modify
it under the terms of the respective public license published by the
Free Software Foundation and included with the repository within
which this application is contained.

This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

---

## Description

<br>This Jupyter notebook provides an application to compute and plot tropical cyclone (TC) multi-scale intensity attributes as decribed in Vukicevic et. al., [2014].</br>

## Author

<br>Henry R. Winterbottom; 19 March 2023</br>

---

<b>Licensing:</b> GNU Lesser General Public License v2.1<br>
<b>Maintainer:</b> Henry R. Winterbottom (henry.winterbottom@noaa.gov)<br>
<b>History:</b><br>
<ul>
    <li>2023-03-19: Henry Winterbottom -- Initial implementation.</li>
</ul>

---

### User Configuration

In [ ]:
# Define the netCDF-formatted file path containing the TC MSI attributes.
ncfile = "/Users/henry.winterbottom/work/tcdiags/14L.TCMSI.nc"

# Define the total number of total-wind field wave number components to plot.
n_wavenum = 2

# Define the TC MSI plotting attributes for the total wind and wave-number 0.
vmax_cint = 5.0
vmax_cmax = 30.0
vmax_cmin = 0.0
vmax_cmap = "jet"

# Define the TC MSI plotting attributes for the wave-number 1 - wave-number N fields.
wn_cint = 1.0
wn_cmax = 5.0
wn_cmin = -5.0
wn_cmap = "seismic"

# Define the TC MSI attributes stored as netCDF global attributes; 
# this dictionary contains the global attribute name and the corresponding units.
tcmsi_gattrs_dict = OrderedDict({"lat_0_deg":{"name": "Center Latitude", "units": "degrees"},
                                 "lon_0_deg":{"name": "Center Longitude", "units": "degrees"},
                                 "vmax_mps": {"name": "Maximum 10-meter Wind Speed", "units": "m/s"},
                                 "wn0_msi_mps": {"name": "Wavenumber-0 Maximum Wind Speed", "units": "m/s"},
                                 "wn1_msi_mps": {"name": "Wavenumber-1 Maximum Wind Speed", "units": "m/s"},
                                 "wn0p1_msi_mps": {"name": "Wavenumbers (0+1) Maximum Wind Speed", "units": "m/s"},
                                 "epsi_msi_mps": {"name": "Residual Wavenumber Maximum Wind Speed", "units": "m/s"},
                                 "lat_rmw_deg": {"name": "Radius of Maximum Wind Latitude", "units": "degrees"},
                                 "lon_rmw_deg": {"name": "Radius of Maximum Wind Longitude", "units": "degrees"},
                                })

### Plot the Tropical Cyclone Multi-scale Intensity Attributes

In [ ]:
# Build a table containing the TC MSI attributes.
header = ["TC MSI Attributes", "Value (Units)"]

table = []

for tcmsi_attr in tcmsi_gattrs_dict:
    value = netcdf4_interface.ncreadattr(ncfile=ncfile, ncattrname=tcmsi_attr)
    name = parser_interface.dict_key_value(dict_in=tcmsi_gattrs_dict[tcmsi_attr],
                                           key="name", no_split=True)
    units = parser_interface.dict_key_value(dict_in=tcmsi_gattrs_dict[tcmsi_attr],
                                           key="units", no_split=True)
    
    msg = [name, f"{value} ({units})"]
    table.append(msg)
    
# Display the table.
table = tabulate(table, header, tablefmt="html", numalign=("center", "center"),
                colalign=("center","center")) 
display(HTML(table))

# Collect the grid attributes.
azimuth = netcdf4_interface.ncreadvar(ncfile=ncfile, ncvarname="azimuth")
radial = netcdf4_interface.ncreadvar(ncfile=ncfile, ncvarname="radial")

# Define the plotting attributes.
levels = numpy.linspace(vmax_cmin, vmax_cmax, 255)

# Plot the total wind field.
(fig, ax) = plt.subplots(subplot_kw={'projection': 'polar'})
total_wind = netcdf4_interface.ncreadvar(ncfile=ncfile, ncvarname="total_wind")
plot = ax.contourf(azimuth, radial, total_wind, levels=levels, cmap=vmax_cmap)
ticks = numpy.arange(vmax_cmin, (vmax_cmax + 0.01), vmax_cint)
plt.colorbar(plot, orientation="horizontal", ticks=ticks, pad=0.1,
             aspect=50, label="Total Wind Speed (m$^{-1}$)")
plt.savefig("tcwnmsi.total_wind.png", dpi=500, transparent=True, bbox_inches="tight")
plt.show()

# Plot the wave-number 0 wind field.
(fig, ax) = plt.subplots(subplot_kw={'projection': 'polar'})
wn0 = netcdf4_interface.ncreadvar(ncfile=ncfile, ncvarname="wn0")
plot = ax.contourf(azimuth, radial, wn0, levels=levels, cmap=vmax_cmap)
ticks = numpy.arange(vmax_cmin, (vmax_cmax + 0.01), vmax_cint)
plt.colorbar(plot, orientation="horizontal", ticks=ticks, pad=0.1,
             aspect=50, label="Wavenumber-0 Wind Speed (m$^{-1}$)")
plt.savefig("tcwnmsi.wn0_wind.png", dpi=500, transparent=True, bbox_inches="tight")
plt.show()

# Plot the higher-order wavenumber structures.
levels = numpy.linspace(wn_cmin, wn_cmax, 255)

for wn in range(1, n_wavenum):
    (fig, ax) = plt.subplots(subplot_kw={'projection': 'polar'})
    wnfield = netcdf4_interface.ncreadvar(ncfile=ncfile, ncvarname=f"wn{wn}")
    plot = ax.contourf(azimuth, radial, wnfield, levels=levels, cmap=wn_cmap)
    ticks = numpy.arange(wn_cmin, (wn_cmax + 0.01), wn_cint)
    plt.colorbar(plot, orientation="horizontal", ticks=ticks, pad=0.1,
                 aspect=50, label=f"Wavenumber-{wn} Wind Speed (m$^{-1}$)")
    plt.savefig(f"tcwnmsi.wn{wn}_wind.png", dpi=500, transparent=True, bbox_inches="tight")
    plt.show()